> # **Assignment 1**

**CSCN8010 Foundations of Machine Learning Frameworks**

**Student Name: Hasyashri Bhatt**

**Student Number : 9028501**

**Practical Lab 2 : Multivariate Linear Regression, Non-Parametric Models and Cross-Validation**

---
**Introduction**

---

In this assignment, we’re working with a diabetes dataset that contains health measurements of 442 patients.The data includes 10 baseline health factors and a target that shows disease progression after a year.This is a well-known dataset frequently used to explore regression and non-parametric methods.

Using this, we aim to better understand the relationship between health factors and diabetes progression.



---

**Objective**

---

The main objective of this assignment is to:

- Analyze and explore the diabetes dataset, understanding its structure and relationships.
- Train regression and non-parametric models — polynomial regression, decision trees, kNN — to predict diabetes progression based on baseline health factors.
- Evaluate and compare models using appropriate metrics (R-squared, Mean Absolute Error, and Mean Absolute Percentage Error).
- Identify which model performs best in making accurate predictions.
- Draw conclusions about the performance, limitations, and potential applications of these models in a healthcare context.

Ultimately, this will help us appreciate how regression and non-parametric methods can aid doctors in identifying high-risk diabetes patients and tailoring their care plans.

---

**1. First we need to import necessary libraries and packages for the further process.**

In [ ]:
import pandas as pd 
import numpy as np
